In [1]:
from pystac_client import Client
import requests
import rasterio

In [19]:
url = 'https://earth-search.aws.element84.com'
version = 'v0'
year = '2019'
datetime = f'{year}-07-01T00:00:00Z/{year}-07-31T23:59:59Z'
collections = ['sentinel-s2-l2a-cogs' ]
query = {
  "eo:cloud_cover": {
    "lte": 10
  }
}
aoi = {
  "type": "Feature",
  "properties": {},
  "geometry": {
    "coordinates": [
      [
        [
          -55.689091261999664,
          5.60860189900589
        ],
        [
          -55.566872617101296,
          6.119855660236823
        ],
        [
          -56.10902199062491,
          6.013902850144206
        ],
        [
          -57.06483446995763,
          6.147898491292224
        ],
        [
          -57.54744142673495,
          6.46872619646804
        ],
        [
          -58.07392174321963,
          6.9760239314053365
        ],
        [
          -58.39983812961492,
          7.401979990778074
        ],
        [
          -59.22092805333307,
          8.266203962607278
        ],
        [
          -59.54890801589994,
          7.874021129644646
        ],
        [
          -59.1736243085698,
          7.430127505109084
        ],
        [
          -58.85182950113523,
          6.929711853279869
        ],
        [
          -58.026083082992145,
          6.206640935191217
        ],
        [
          -57.17912820908509,
          5.635994244239754
        ],
        [
          -56.26397566934364,
          5.441453990199037
        ],
        [
          -55.9694979522222,
          5.500029201680249
        ],
        [
          -55.689091261999664,
          5.60860189900589
        ]
      ]
    ],
    "type": "Polygon"
  }
}
limit = 1
sort = [
  {
    "field": "eo:cloud_cover",
    "direction": "asc"
  }
]

In [20]:
params = {
  "collections": collections,
  "intersects": aoi['geometry'],
  "datetime": datetime,
  "query": query,
  "limit": limit
}

In [21]:
catalog = Client.open(f'{url}/{version}')
search = catalog.search(**params, max_items=limit)

found = search.matched()
if found is not None:
  print(f'Scenes found: {found}')


scenes = [scene.to_dict() for scene in search.get_items()]
print(f'Scenes fetched: {len(scenes)}')

Scenes found: 13
Scenes fetched: 1


In [5]:
response = requests.post(f'{url}/{version}/search', json=params)
scenes = []
if response.ok:
  json_response = response.json()
  features = json_response.get('features', [])

  for f in features:
    scenes.append(f)

print(f'Scenes fetched: {len(scenes)}') 

Scenes fetched: 1


In [19]:
collections = [
  {
    "id": "sentinel-s2-l2a-cogs",
    "bands": ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B11']
  }
]

def download_image(url):
  urlsplit = url.split('/')
  dst_file = f'data/{urlsplit[-2]}_{urlsplit[-1]}'
  with rasterio.open(url) as img:
    print(f'writing {dst_file}')
    with rasterio.open(dst_file, 'w', **img.profile) as dst:
      dst.write(img.read())

  

for scene in scenes:
  collection_items = list(filter(lambda collection: collection['id'] == scene['collection'], collections))
  if len(collection_items) > 0:
    collection = collection_items[0]
    bands = collection['bands']
    for band in bands:
      image_url = scene['assets'][band]['href']
      download_image(image_url)



In [9]:
!curl -O -J https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/20/P/RQ/2019/7/S2B_20PRQ_20190730_0_L2A/B01.tif

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4965k  100 4965k    0     0  77676      0  0:01:05  0:01:05 --:--:--  107k4408      0  0:01:18  0:00:40  0:00:38 26645:00:13  177k77266      0  0:01:05  0:01:05 --:--:-- 92778


In [10]:
!curl -O -J https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/20/P/RQ/2019/7/S2B_20PRQ_20190730_0_L2A/B02.tif

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  170M  100  170M    0     0   101k      0  0:28:34  0:28:34 --:--:--  362k 0 42967      0  1:09:21  0:00:30  1:08:51 17164  0  1:35:53  0:01:16  1:34:37 31403    0  1:07:56  0:01:42  1:06:14 334891:09:43  0:02:01  1:07:42 47493  0:58:07  0:02:37  0:55:30 263960  52183      0  0:57:06  0:02:41  0:54:25 83417154k 0:03:39  0:43:22  109k2272      0  0:41:13  0:03:47  0:37:26  281k86400    0  0:36:25  0:04:09  0:32:16  149k88878      0  0:33:31  0:06:24  0:27:07 586827M    0     0  94124      0  0:31:39  0:06:49  0:24:50 77497 0:06:52  0:24:47 82807  0:07:25  0:23:20 97728   0  95832      0  0:31:05  0:08:12  0:22:53 9176901  0:08:19  0:22:42 89686  0  95531      0  0:31:11  0:08:28  0:22:43 734660  95557      0  0:31:10  0:08:32  0:22:38 96576M    0     0  96403      0  0:30:54  0:08:46  0:22:08  114k  96474      0  0:30:53  0:08:

In [11]:
!curl -O -J https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/20/P/RQ/2019/7/S2B_20PRQ_20190730_0_L2A/B03.tif

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  171M  100  171M    0     0  91076      0  0:32:59  0:32:59 --:--:--  125k2  0:00:32  0:16:00 375099  0:00:34  0:16:45 25154  142k      0  0:20:37  0:00:47  0:19:50 683335  0:00:53  0:20:42 854670:26:19  0:01:21  0:24:58 93398  0   119k      0  0:24:28  0:01:51  0:22:37   99k  0:02:39  0:24:26 38723  107k      0  0:27:18  0:02:42  0:24:36 5305143  0:03:00  0:24:43 657553k      0  0:28:25  0:03:35  0:24:50 427710k      0  0:29:16  0:03:45  0:25:31 246724  0:03:49  0:25:45 42903 0  99730      0  0:30:08  0:03:58  0:26:10 5622597972      0  0:30:40  0:04:11  0:26:29 799968  0:04:26  0:26:52 80613    0  0:30:59  0:04:51  0:26:08 8002896122      0  0:31:16  0:05:27  0:25:49  131k102k      0  0:28:30  0:05:51  0:22:39  312k8:02  0:05:54  0:22:08  301k27  0:06:01  0:21:26  220k.3M    0     0   107k      0  0:27:23  0:06:06  0:21:17  

In [12]:
!curl -O -J https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/20/P/RQ/2019/7/S2B_20PRQ_20190730_0_L2A/B04.tif

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 77  168M   77  131M    0     0  99304      0  0:29:40  0:23:06  0:06:34 95138 02  0:00:21  0:15:51  122k 0:18:28  0:00:32  0:17:56  115k   0   189k      0  0:15:11  0:01:00  0:14:11  169k15:19  0:01:05  0:14:14  167k    0  0:16:52  0:01:18  0:15:34 90089  0:01:31  0:16:13  113k4k      0  0:19:51  0:01:58  0:17:53  159k   0   145k      0  0:19:49  0:02:00  0:17:49  158kk      0  0:20:36  0:02:10  0:18:26 59580 0:02:27  0:19:53 34934 0:02:32  0:20:16 32894     0  0:22:53  0:02:33  0:20:20 377460   122k      0  0:23:32  0:02:38  0:20:54 329630:23:42  0:02:40  0:21:02 45838 0:02:45  0:21:19 45705  0:02:53  0:22:01 46326  0:03:12  0:22:58 75927   108k      0  0:26:34  0:03:28  0:23:06  107k    0  0:26:34  0:03:29  0:23:05  102k 0   107k      0  0:26:49  0:03:32  0:23:17 7128807k      0  0:26:53  0:03:33  0:23:20 559183:44  0:23:47 343

In [13]:
!curl -O -J https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/20/P/RQ/2019/7/S2B_20PRQ_20190730_0_L2A/B05.tif

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 44.8M  100 44.8M    0     0  78837      0  0:09:56  0:09:56 --:--:--  105k0:17:14  0:01:04  0:16:10 69079  0  0:13:44  0:01:15  0:12:29  195k0      0  0:10:23  0:01:53  0:08:30  217k   0  0:10:07  0:01:54  0:08:13  250k  0  0:09:28  0:02:32  0:06:56 56877      0  0:11:01  0:04:21  0:06:40 48820  70261      0  0:11:09  0:04:36  0:06:33 591048  0:04:49  0:06:19 66589      0  0:11:10  0:04:53  0:06:17 5238170712      0  0:11:05  0:05:12  0:05:53  122k 73009      0  0:10:44  0:05:21  0:05:23  128k5:25  0:05:21 55985:51  0:04:43  113k30 61042 0  77606      0  0:10:05  0:06:49  0:03:16 349830:07:24  0:02:38 82683      0  0:10:04  0:07:27  0:02:37 4548865      0  0:09:48  0:08:09  0:01:39  102k7  0:08:52  0:01:05 8790192      0  0:09:57  0:09:02  0:00:55 72807


In [14]:
!curl -O -J https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/20/P/RQ/2019/7/S2B_20PRQ_20190730_0_L2A/B06.tif

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  3 44.7M    3 1612k    0     0  84407      0  0:09:16  0:00:19  0:08:57 75645k      0  0:06:58  0:00:06  0:06:52  128k6k      0  0:07:10  0:00:07  0:07:03  115k^C


In [15]:
!curl -O -J https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/20/P/RQ/2019/7/S2B_20PRQ_20190730_0_L2A/B07.tif

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 44.9M  100 44.9M    0     0  49275      0  0:15:55  0:15:55 --:--:-- 6005444k      0  0:05:17  0:00:44  0:04:33 29944  0:00:59  0:05:24 76390  0:07:37  0:01:28  0:06:09 36266  0  0:07:56  0:01:34  0:06:22 34913      0  0:09:27  0:02:13  0:07:14 31708 0:09:37  0:02:20  0:07:17 585545  0:02:45  0:07:40 21732  0  72408      0  0:10:50  0:03:10  0:07:40 68253 0     0  63574      0  0:12:20  0:05:13  0:07:07 4957904 55705  0:05:54  0:06:33 403642:41  0:06:18  0:06:23 44910 0:13:11  0:06:53  0:06:18 16825  0:07:02  0:06:12 48219  0:07:03  0:06:12 41373      0  0:13:31  0:07:27  0:06:04 4490343  0:07:39  0:06:04 2745796      0  0:13:46  0:07:47  0:05:59 48200 53635      0  0:14:37  0:09:15  0:05:22 36609 0:09:30  0:05:19 2745028      0  0:15:05  0:09:53  0:05:12 32136360  51915      0  0:15:07  0:09:59  0:05:08 460151841      0  0:15

In [ ]:
!curl -O -J https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/20/P/RQ/2019/7/S2B_20PRQ_20190730_0_L2A/B08.tif

In [18]:
!curl -O -J https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/20/P/RQ/2019/7/S2B_20PRQ_20190730_0_L2A/B09.tif

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5509k  100 5509k    0     0   849k      0  0:00:06  0:00:06 --:--:-- 1102k


In [17]:
!curl -O -J https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/20/P/RQ/2019/7/S2B_20PRQ_20190730_0_L2A/B11.tif

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 43.3M  100 43.3M    0     0  2655k      0  0:00:16  0:00:16 --:--:-- 3263k


In [2]:
!curl -O -J https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/20/P/RQ/2019/7/S2B_20PRQ_20190730_0_L2A/B12.tif

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 42.6M  100 42.6M    0     0  35555      0  0:20:57  0:20:57 --:--:-- 26624 0:00:31  0:16:32 55428 40502      0  0:18:23  0:00:49  0:17:34 27417:56 306990:11 21259      0  0:22:03  0:01:34  0:20:29 41596:01:50  0:19:26 518730:15:54  0:03:18  0:12:36 208729  0:03:26  0:12:43 31179  0:03:47  0:12:32 49294  44194      0  0:16:51  0:05:04  0:11:47 20906   0  0:16:55  0:05:07  0:11:48 272472  0:05:30  0:11:32 15175   0  0:17:40  0:05:52  0:11:48 24415     0  0:18:51  0:07:26  0:11:25 3407911  0:08:00  0:11:11 28421 0:19:33  0:08:35  0:10:58 170265  0:08:56  0:10:49 17314      0  0:19:48  0:09:16  0:10:32 2403437371      0  0:19:55  0:09:26  0:10:29 27381153      0  0:19:31  0:10:36  0:08:55 52078719:31  0:11:47  0:07:44 209440  0:19:31  0:11:49  0:07:42 32397 38152      0  0:19:31  0:12:16  0:07:15 277860:12:26  0:07:11 27385 0  0:1

In [1]:
import os

In [6]:
folder = os.curdir # 'mapping_mangrove/data'

for file in os.listdir(folder):
  if not file.endswith('.tif'):
    continue
  print(file)
  old_name = os.path.join(folder, file)
  new_name = os.path.join(folder, 'S2B_20PRQ_20190730_0_L2A_' + file)
  os.rename(old_name, new_name)

B12.tif


In [11]:
res = os.listdir(folder)
print(res)

['S2B_20PRQ_20190730_0_L2A_B09.tif', 'S2B_20PRQ_20190730_0_L2A_B08.tif', 'S2B_20PRQ_20190730_0_L2A_B11.tif', 'S2B_20PRQ_20190730_0_L2A_B05.tif', 'S2B_20PRQ_20190730_0_L2A_B04.tif', 'S2B_20PRQ_20190730_0_L2A_B06.tif', 'S2B_20PRQ_20190730_0_L2A_B07.tif', 'S2B_20PRQ_20190730_0_L2A_B03.tif', 'S2B_20PRQ_20190730_0_L2A_B02.tif', 'S2B_20PRQ_20190730_0_L2A_B01.tif']


In [25]:
scene = scenes[0]
image_feature = scene['geometry']
print(image_feature)

{'type': 'Polygon', 'coordinates': [[[-59.283659390194934, 8.03675375646586], [-60.24876781372564, 8.044356186854495], [-60.030820917840686, 9.034711359163778], [-59.27405212273216, 9.027803658954483], [-59.283659390194934, 8.03675375646586]]]}


In [39]:
from shapely.geometry import shape

shp_polygon = shape(image_feature)
print(shp_polygon.geometryType)

from ipyleaflet import Map, WKTLayer
wkt_layer = WKTLayer(wkt_string=shp_polygon.wkt)
m = Map(zoom=6, center=(7.430127505109084, -59.1736243085698))
m.add_layer(wkt_layer)

m

<bound method BaseGeometry.geometryType of <shapely.geometry.polygon.Polygon object at 0x7fb77003ae80>>


Map(center=[7.430127505109084, -59.1736243085698], controls=(ZoomControl(options=['position', 'zoom_in_text', …